# Proyecto 2
Insertar alguna descripción

## Implementación del algoritmo con Dynamic Programming
El resultado del algoritmo es una matriz donde el eje vertical (eje y) representa las denominaciones de las monedas, ordenadas de menor a mayor. El valor en una casilla con coordenadas (x, y) representa el número mínimo de monedas necesarias para formar el valor x utilizando todas las monedas con denominaciones menores o iguales a y. Por lo tanto, el resultado final del algoritmo es el número ubicado en la esquina inferior derecha de la matriz. Este número indica el mínimo número de monedas necesarias para formar el valor deseado con las monedas disponibles.

In [18]:
def init_table(coins: [int], amount: int):
    table = [[0]*(amount+1) for _ in range(len(coins))]

    for i in range(len(table[0])):
        table[0][i] = i

    return table

# the amounts MUST include a 1 cent coin
# the coins list MUST be sorted
def coin_change(coins: [int], amount: int):    
    table = init_table(coins, amount)

    for row in range(1, len(coins)):
        for col in range(amount+1):
            if col < coins[row]:
                table[row][col] = table[row-1][col]
            else:
                table[row][col] = min(table[row-1][col], 1 + table[row][col-coins[row]])

    return table


def print_result(coins: [int], amount: int):
    table = coin_change(coins, amount)

    print("x", end="  ")
    for i in range(amount+1):
        print(i, end="  ")
    print()

    for i in range(len(table)):
        print(coins[i], end=" ")
        print(table[i])

coins = [1,3,4,5]
amount = 7
table = print_result(coins, amount)


x  0  1  2  3  4  5  6  7  
1 [0, 1, 2, 3, 4, 5, 6, 7]
3 [0, 1, 2, 1, 2, 3, 2, 3]
4 [0, 1, 2, 1, 1, 2, 2, 2]
5 [0, 1, 2, 1, 1, 1, 2, 2]


### Análisis empírico

## Implementación del algoritmo con Divide and Conquer

In [ ]:
print()

### Análisis empírico